# Signing and verifying a block header 

## 1. Creating Encryption Key pair

In [1]:
def generate_RSA(bits=2048):
    '''
    Generate an RSA keypair with an exponent of 65537 in PEM format
    param: bits The key length in bits
    Return private key and public key
    '''
    from Crypto.PublicKey import RSA 
    new_key = RSA.generate(bits, e=65537) 
    public_key = new_key.publickey().exportKey("PEM") 
    private_key = new_key.exportKey("PEM") 
    return private_key, public_key


In [3]:
from Crypto.PublicKey import RSA 

alice_priv, alice_pub = generate_RSA()
bob_priv, bob_pub = generate_RSA()

with open("resources/alice_pub.pem", "wb") as file:
    file.write(alice_pub)

with open("resources/alice_priv.pem", "wb") as file:
    file.write(alice_priv)

with open("resources/bob_pub.pem", "wb") as file:
    file.write(bob_pub)

with open("resources/bob_priv.pem", "wb") as file:
    file.write(bob_priv)

## 2. Signing a blocker header using private key

In [7]:
import json 
from Crypto.Hash import SHA

def hash_block(msg=""):
    # Hashes a list of dictionaries
    if type(msg)!=str:
        msg = json.dumps(msg,sort_keys=True)  # If we don't sort keys, we can't guarantee repeatability!
        
    return SHA.new(str.encode(msg))

In [8]:
# Load resources
with open("resources/alice_priv.pem", "rb") as file:
    alice_priv = RSA.importKey(file.read())
    
with open('resources/alice_pub.pem', "rb") as file:
    alice_pub = RSA.importKey(file.read())

with open("resources/bob_priv.pem", "rb") as file:
    bob_priv = RSA.importKey(file.read())
    
with open('resources/bob_pub.pem', "rb") as file:
    bob_pub = RSA.importKey(file.read())

In [9]:
from Crypto.Signature import PKCS1_v1_5

txns = [
    {"Bob": -50, "Alice": 50},
    {"Bob": -25, "Alice": 25}
]

block_header = {u'blockNumber':0, u'parentHash':None, u'txn':txns}
hash_obj = hash_block(txns)
signer = PKCS1_v1_5.new(bob_priv)
signature = signer.sign(hash_obj)

signature

b'&\x0b\xb0\xd3\xf6\xfc\\\xa6\xb6\xf07Z\x131t&%L\xe4N\xa4\xc8\x80\x90\xf0h\xa2\xc9\x02p\x1d\x1d7\x89\xa3\x87H\x02\\\x05\xeac\xec\xbc\x1c\x9a\xef\xbc\x12X\x07\x9d\n|\xab\x97\xe0\x0f]\xb0\xd2\tE\xe9\x04\t\xf3"U\xa0Q\xc5\x86\xfe\x7f\x17\xa6$\x90l\xf8\xa0\xdb\x1a\x92\x92\x02\xb2PHO\xa0\xbba\\\x9e\x81Q\x1e)p1A.\xf1\x85\xfeS\xe3\x90\x96\xa6\xb6<@Xm\x9d\x88\xc8\x17\xa9\x93\xa1\xcf\xa3\xd9\xf1\xacF\xdd\xd6\x0bc\xad\xb5\x15V\xee\xac>\xe1\xf5\xd7\xc3\xf3\x84\xe8\xf8\xbe1\xa3\xb6\xed\xb8x5\xdd\xf2\xb0\x04d/\xaeZR)L\xb8\xab\xc0c0X\xd0\xd4]\xb7\xbb\x0e\x81cXA\x85\xf6\x9bS\xdf\x8bQd:\xf3\xfa$\xa8\xc5\x96\xb8\xf3oq\xa7\x82\xe9\xac/\xed]\xda\xfb\xf3\x1d\x03\xcc\x06H1\x94\xf2T\xa0\xf3w1\xb9\xa1\xcd\x07+f[\xf8\xad\xd8\xb4\x98\xe4\xb7K\x07e\xa4\rA\xd2B\x1f\xca\xcf\xdc\xfb\xf0\x03s'

In [11]:
import pickle

with open("resources/signature.pkl", "wb") as file: 
    pickle.dump(signature, file)

## 3. Verifying a block header using public key

In [14]:
from Crypto.Signature import PKCS1_v1_5

txns = [
    {"Bob": -50, "Alice": 50},
    {"Bob": -25, "Alice": 25}
]

block_header = {u'blockNumber':0, u'parentHash':None, u'txn':txns}
hash_obj = hash_block(txns)

verifier = PKCS1_v1_5.new(bob_pub)

result = verifier.verify(hash_obj, signature)
